Лабораторная 4 Храмцова Елизавета БИВТ-22-СП-1

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = r'C:\Users\eliza\Downloads\data (1).csv'  
data = pd.read_csv(file_path)
data.head()

,EducationSector,IndividualProject,Age,Gender,City,Influenced,Perseverance,DesireToTakeInitiative,Competitiveness,SelfReliance,StrongNeedToAchieve,SelfConfidence,GoodPhysicalHealth,MentalDisorder,KeyTraits,ReasonsForLack,y
0,Engineering Sciences,No,19,Male,Yes,No,2,2,3,3,2,2,3,Yes,Passion,NaN,1
1,Engineering Sciences,Yes,22,Male,No,Yes,3,3,3,4,4,3,4,Yes,Vision,Just not interested! (Want to work in the corp...,0
2,Engineering Sciences,No,18,Male,Yes,No,3,4,3,3,3,4,4,No,Passion,Not willing to start a venture in India and wa...,0
3,Engineering Sciences,Yes,20,Male,Yes,Yes,3,3,3,3,4,3,3,No,Resilience,Not able to take a Financial Risk,0
4,Engineering Sciences,Yes,19,Male,Yes,Yes,2,3,3,3,4,3,2,Yes,Vision,NaN,1


##  Делаем предобработку данных

In [11]:
from sklearn.model_selection import train_test_split

X = data.drop('y', axis=1)
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### проверка пропусков


In [12]:
data.isnull().sum()

EducationSector            0
IndividualProject          0
Age                        0
Gender                     0
City                       0
Influenced                 0
Perseverance               0
DesireToTakeInitiative     0
Competitiveness            0
SelfReliance               0
StrongNeedToAchieve        0
SelfConfidence             0
GoodPhysicalHealth         0
MentalDisorder             0
KeyTraits                  0
ReasonsForLack            91
y                          0
dtype: int64

In [13]:
#data.fillna({"ReasonsForLack": "unknown"})

In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

categorical_cols = data.select_dtypes(include=['object']).columns

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

Кодировка

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

X_train_label_encoded = X_train.copy()
X_test_label_encoded = X_test.copy()

for column in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train_label_encoded[column] = le.fit_transform(X_train_label_encoded[column])
    
    # Проверяем, есть ли в тестовом наборе значения, которых нет в обучающем
    unseen_labels = set(X_test_label_encoded[column]) - set(le.classes_)
    if unseen_labels:
        print(f"Невидимые метки в тестовом наборе для {column}: {unseen_labels}")
    
    # Применяем transform только к известным меткам
    X_test_label_encoded[column] = le.transform(X_test_label_encoded[column].where(X_test_label_encoded[column].isin(le.classes_)))
    
    label_encoders[column] = le

NameError: name 'X_train' is not defined

In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Пример кодирования категориальных переменных
label_encoders = {}
X_train_label_encoded = X_train.copy()
X_test_label_encoded = X_test.copy()

for column in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train_label_encoded[column] = le.fit_transform(X_train_label_encoded[column])
    X_test_label_encoded[column] = le.transform(X_test_label_encoded[column])

    label_encoders[column] = le

# Проверка на NaN
print("NaN в обучающем наборе:\n", X_train_label_encoded.isnull().sum())
print("NaN в тестовом наборе:\n", X_test_label_encoded.isnull().sum())

# Заполнение NaN средним значением (или другим методом)
X_train_label_encoded.fillna(X_train_label_encoded.mean(), inplace=True)
X_test_label_encoded.fillna(X_test_label_encoded.mean(), inplace=True)

# Применение StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_label_encoded)
X_test_scaled = scaler.transform(X_test_label_encoded)

ValueError: y contains previously unseen labels: 'Language and Cultural Studies'

## Dataset и Dataloader


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if isinstance(self.features, torch.Tensor):
            feature = self.features[idx]
        else:
            feature = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)

        if isinstance(self.targets, torch.Tensor):
            target = self.targets[idx]
        else:
            target = torch.tensor(self.targets.iloc[idx], dtype=torch.float32)

        return feature, target

train_dataset = CustomDataset(torch.tensor(X_train_scaled, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
test_dataset = CustomDataset(torch.tensor(X_test_scaled, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [2]:
import torch.nn as nn
import torch.optim as optim
criterion = nn.MSELoss()

model1 = nn.Sequential(
    nn.Linear(X_train_label_encoded.shape[1], 1)
)
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)

model2 = nn.Sequential(
    nn.Linear(X_train_label_encoded.shape[1], 10),
    nn.ReLU(),
    nn.Linear(10, 1)
)
optimizer2 = optim.Adam(model2.parameters(), lr=0.01)

model3 = nn.Sequential(
    nn.Linear(X_train_label_encoded.shape[1], 20),
    nn.ReLU(),
    nn.Linear(20, 15),
    nn.ReLU(),
    nn.Linear(15, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
)
optimizer3 = optim.Adam(model3.parameters(), lr=0.01)

def train_model(model, optimizer, criterion, train_loader, test_loader, epochs=100):
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for features, targets in train_loader:
            outputs = model(features)
            loss = criterion(outputs.flatten(), targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_losses.append(train_loss / len(train_loader))
        test_loss = 0.0
        with torch.no_grad():
            for features, targets in test_loader:
                outputs = model(features)
                loss = criterion(outputs.flatten(), targets)
                test_loss += loss.item()
        test_losses.append(test_loss / len(test_loader))
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")
    return train_losses, test_losses

print("Обучение однослойной нейросети с SGD...")
train_losses1, test_losses1 = train_model(model1, optimizer1, criterion, train_loader, test_loader, epochs=100)

print("\nОбучение нейросети с одним скрытым слоем с Adam...")
train_losses2, test_losses2 = train_model(model2, optimizer2, criterion, train_loader, test_loader, epochs=100)

print("\nОбучение нейросети с тремя скрытыми слоями с Adam...")
train_losses3, test_losses3 = train_model(model3, optimizer3, criterion, train_loader, test_loader, epochs=100)

NameError: name 'X_train_label_encoded' is not defined

## Рассчет loss

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
plt.plot(train_losses1, label="train 1 с оптимизатором SGD", linestyle='--')
plt.plot(test_losses1, label="test 1 с оптимизатором SGD")
plt.plot(train_losses2, label="train 2 c одним скрытым слоем и Adam", linestyle='--')
plt.plot(test_losses2, label="test 2 c одним скрытым слоем и Adam")
plt.plot(train_losses3, label="train 3 c 3+ скрытыми + Adam", linestyle='--')
plt.plot(test_losses3, label="test 3 c 3+ скрытыми + Adam")
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Learning Curves")
plt.legend()
plt.show()

## Рассчет метрики MAE

In [ ]:
from sklearn.metrics import mean_absolute_error

def calculate_mae(model, loader):
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for features, targets in loader:
            preds = model(features).flatten()
            all_preds.extend(preds.numpy())
            all_targets.extend(targets.numpy())
    return mean_absolute_error(all_targets, all_preds)

mae_train1 = calculate_mae(model1, train_loader)
mae_test1 = calculate_mae(model1, test_loader)

mae_train2 = calculate_mae(model2, train_loader)
mae_test2 = calculate_mae(model2, test_loader)

mae_train3 = calculate_mae(model3, train_loader)
mae_test3 = calculate_mae(model3, test_loader)

print("Однослойная модель (SGD):")
print(f"Train MAE: {mae_train1}, Test MAE: {mae_test1}")

print("\nМодель с одним скрытым слоем (Adam):")
print(f"Train MAE: {mae_train2}, Test MAE: {mae_test2}")

print("\nМодель с тремя скрытыми слоями (Adam):")
print(f"Train MAE: {mae_train3}, Test MAE: {mae_test3}")